In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tsdm.util import (
    float_gcd,
    time_gcd,
    regularity_coefficient,
    is_regular,
    is_quasiregular,
)
from tsdm.datasets import Electricity
import numpy as np

In [ ]:
from numba import jit


def float_gcd(x, rtol=1e-08, atol=1e-08):
    x = np.asanyarray(x)
    x = np.abs(x).flatten()

    #     @jit
    def _float_gcd(x: np.ndarray):
        print(x, end="")
        n = len(x)
        if n == 1:
            print("->", x[0])
            return x[0]
        if n == 2:
            t = np.min(x)
            while x[1] >= rtol * t + atol:
                x[0], x[1] = x[1], x[0] % x[1]
            print("->", x[0])
            return x[0]
        if n >= 3:
            out = np.full(2, np.nan)
            out[0] = _float_gcd(x[: n // 2])
            out[1] = _float_gcd(x[n // 2 :])
            return _float_gcd(out)

    return _float_gcd(x)

In [ ]:
X = Electricity.dataset

In [ ]:
x = np.random.randn(100) * 10
x = np.array([2, 4, 4 + 2**-10, 6])

In [ ]:
m, e = np.frexp(x)

z = x * 2 ** int(52 - np.min(e))
assert np.allclose(z, np.rint(z))

gcd = np.gcd.reduce(np.rint(z).astype(int))
gcd = gcd * 2 ** int(np.min(e) - 52)
gcd

In [ ]:
float_gcd(x, rtol=0, atol=2**-8)

In [ ]:
y = [4.00097656, 6.0]

for k in range(10):
    y[0], y[1] = y[1], y[0] % y[1]
    print(y)

In [ ]:
# float: (-1)^sign (1+ sum_{i=1}^52 b[52-i] 2^{-i}) * 2^(e-1023)"


def float2ints(x) -> tuple[int, int, int]:
    x = np.array(x, dtype=np.float64)
    m, e = np.frexp(x)
    m = np.array(m)
    e = np.array(e).astype(int)
    s, m = np.sign(m).astype(int), np.rint(np.abs(m) * 2**52).astype(int)
    return s.squeeze(), m.squeeze(), e.squeeze()


def ints2float(s: int, m: int, e: int) -> float:
    return np.ldexp(s * (m * 2**-52), e)